### California Fire Events 

In [1]:
import pandas as pd
import geopandas as gpd
import shapely
import ee, geemap
import json

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
# Parameters for data acquisition
CLOUD_THRESHOLD = 20
PRE_FIRE_DAYS = 60  # Days before fire to include
POST_FIRE_DAYS = 30  # Days after containment to include


# Parameters for export
TARGET_PIXELS = 224
SCALE = 20  # Meters per pixel
# Approximate meters per degree at equator
# We prefer to large image for cropping
METERS_PER_DEGREE = 111320 

In [7]:
in_file = f"../../data/California_Historic_Fire_Perimeters.geojson"

In [8]:
fire_gdf = gpd.read_file(in_file)

In [9]:
fire_df = pd.DataFrame(fire_gdf)
fire_df["ALARM_DATE_ISO"] = pd.to_datetime(fire_df.ALARM_DATE, format='%a, %d %b %Y %H:%M:%S %Z').dt.tz_convert('UTC').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
fire_df["CONT_DATE_ISO"]  = pd.to_datetime(fire_df.CONT_DATE, format='%a, %d %b %Y %H:%M:%S %Z').dt.tz_convert('UTC').dt.strftime('%Y-%m-%dT%H:%M:%SZ')

In [10]:
medium_fires = fire_df[
    (fire_df.GIS_ACRES < 10000) & 
    (fire_df.GIS_ACRES > 1000) & 
    (fire_df.DECADES == "2020-January 2025") &
    (fire_df.ALARM_DATE_ISO.notna()) &
    (fire_df.CONT_DATE_ISO.notna()) ]

In [11]:
small_fires = fire_df[
    (fire_df.GIS_ACRES < 1000) & 
    (fire_df.GIS_ACRES > 100) & 
    (fire_df.DECADES == "2020-January 2025") &
    (fire_df.ALARM_DATE_ISO.notna()) &
    (fire_df.CONT_DATE_ISO.notna()) ]

In [12]:
very_small_fires = fire_df[
    (fire_df.GIS_ACRES < 100) & 
    (fire_df.GIS_ACRES > 0) & 
    (fire_df.DECADES == "2020-January 2025") &
    (fire_df.ALARM_DATE_ISO.notna()) &
    (fire_df.CONT_DATE_ISO.notna()) ]

In [10]:
small_fires.info()

<class 'pandas.core.frame.DataFrame'>
Index: 361 entries, 3 to 1703
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   OBJECTID        361 non-null    int32   
 1   YEAR_           361 non-null    float64 
 2   STATE           361 non-null    object  
 3   AGENCY          361 non-null    object  
 4   UNIT_ID         361 non-null    object  
 5   FIRE_NAME       361 non-null    object  
 6   INC_NUM         361 non-null    object  
 7   ALARM_DATE      361 non-null    object  
 8   CONT_DATE       361 non-null    object  
 9   CAUSE           361 non-null    int32   
 10  C_METHOD        361 non-null    float64 
 11  OBJECTIVE       356 non-null    float64 
 12  GIS_ACRES       361 non-null    float64 
 13  COMMENTS        69 non-null     object  
 14  COMPLEX_NAME    13 non-null     object  
 15  IRWINID         345 non-null    object  
 16  FIRE_NUM        0 non-null      object  
 17  COMPLEX_ID      13 n

In [11]:
medium_fires.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 25 to 1606
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   OBJECTID        133 non-null    int32   
 1   YEAR_           133 non-null    float64 
 2   STATE           133 non-null    object  
 3   AGENCY          133 non-null    object  
 4   UNIT_ID         133 non-null    object  
 5   FIRE_NAME       133 non-null    object  
 6   INC_NUM         133 non-null    object  
 7   ALARM_DATE      133 non-null    object  
 8   CONT_DATE       133 non-null    object  
 9   CAUSE           133 non-null    int32   
 10  C_METHOD        133 non-null    float64 
 11  OBJECTIVE       131 non-null    float64 
 12  GIS_ACRES       133 non-null    float64 
 13  COMMENTS        28 non-null     object  
 14  COMPLEX_NAME    12 non-null     object  
 15  IRWINID         131 non-null    object  
 16  FIRE_NUM        0 non-null      object  
 17  COMPLEX_ID      11 

In [13]:
fire_events = pd.concat([small_fires, medium_fires], axis=0)
len(fire_events)

494

### Data Acquisition from Googel Earth

In [14]:
analysis_regions = []
for idx, fire in fire_events .iterrows():
    fire_geom = fire.geometry
    analysis_regions.append({
        'name': fire.FIRE_NAME,
        'geometry': fire_geom,
        'alarm_date': fire.ALARM_DATE_ISO,
        'containment_date': fire.CONT_DATE_ISO,
        'acres_burned': fire.GIS_ACRES
    })

In [15]:
region = analysis_regions[0]
region_gdf = gpd.GeoDataFrame([region], geometry='geometry', crs='EPSG:4326')
region_ee = geemap.geopandas_to_ee(region_gdf)
region_geom = region_ee.geometry()

In [16]:
bounds = region_geom.bounds()
coord = bounds.coordinates().getInfo()[0]
coord

[[-118.70728067382313, 34.156009687292254],
 [-118.66448187491483, 34.156009687292254],
 [-118.66448187491483, 34.19275500162451],
 [-118.70728067382313, 34.19275500162451],
 [-118.70728067382313, 34.156009687292254]]

In [15]:
def expand_region(region_geom):
    """Expand region bounds if smaller than target crop size in meters."""
    # Extract the coordinates and bounds
    bounds = region_geom.bounds()
    coord = bounds.coordinates().getInfo()[0]
    xmin, ymin = coord[0]
    xmax, ymax = coord[2]

    # Compute the image size
    width_m = (xmax - xmin) * METERS_PER_DEGREE
    height_m = (ymax - ymin) * METERS_PER_DEGREE
    min_size_m = SCALE * TARGET_PIXELS

    # Compute padding if the export image will smaller than target
    pad_x = max(0, (min_size_m - width_m) / METERS_PER_DEGREE / 2)
    pad_y = max(0, (min_size_m - height_m) / METERS_PER_DEGREE / 2)

    # Expand the bounds
    new_bounds = ee.Geometry.Rectangle([xmin - pad_x, ymin - pad_y, xmax + pad_x, ymax + pad_y])
    return new_bounds
    

In [16]:
output_folder = f"dataset"

In [17]:
for region in analysis_regions:
    region_gdf = gpd.GeoDataFrame([region], geometry='geometry', crs='EPSG:4326')
    region_ee = geemap.geopandas_to_ee(region_gdf)
    region_geom = region_ee.geometry()
    print(f"Processing region: {region['name']}")

    # Calculate time window for imagery using ISO
    pre_fire_start = ee.Date(region['alarm_date']).advance(-PRE_FIRE_DAYS, 'day')
    post_fire_end = ee.Date(region['containment_date']).advance(POST_FIRE_DAYS, 'day')

    # Get Sentinel-2 collection for this region and time period
    expand_regions = expand_region(region_geom=region_geom)
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(expand_regions)
        .filterDate(pre_fire_start, post_fire_end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_THRESHOLD))
        .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])
    )
    # Create pre-fire and post-fire composites
    # Clip to expand regions to ensure the minimum crop size
    pre_fire_img = collection.filterDate(pre_fire_start, ee.Date(region['alarm_date'])).median().clip(expand_regions)
    post_fire_img = collection.filterDate(ee.Date(region['containment_date']), post_fire_end).median().clip(expand_regions)

    # Calculate NBR
    pre_fire_nbr = pre_fire_img.normalizedDifference(['B8', 'B12']).rename('NBR')
    post_fire_nbr = post_fire_img.normalizedDifference(['B8', 'B12']).rename('NBR')
    dnbr = pre_fire_nbr.subtract(post_fire_nbr).rename('dNBR')

    # Create fire mask
    fire_mask = ee.Image(0).byte().paint(region_geom, 1).rename("mask").clip(expand_regions)

    # Combine all bands into separate images for smaller exports
    clean_name = region['name'].replace(' ', '_').replace('/', '_')
    alarm_date_short = region['alarm_date'][:10].replace('-', '')

    # Export Pre-fire image 
    pre_fire_visual = pre_fire_img.select(['B2', 'B3', 'B4', 'B8', 'B12'])  
    task1 = ee.batch.Export.image.toDrive(
        image=pre_fire_visual,
        description=f'{clean_name}_{alarm_date_short}',
        folder=f'{output_folder}/pre_fire',
        scale=20,
        region=expand_regions,
        maxPixels=1e10,
        fileFormat='GeoTIFF'
    )
    task1.start()

    # Export Post-fire image 
    post_fire_visual = post_fire_img.select(['B2', 'B3', 'B4', 'B8', 'B12'])  
    task2 = ee.batch.Export.image.toDrive(
        image=post_fire_visual,
        description=f'{clean_name}_{alarm_date_short}',
        folder=f'{output_folder}/post_fire',
        scale=20,
        region=expand_regions,
        maxPixels=1e10,
        fileFormat='GeoTIFF'
    )
    task2.start()

    # Export Fire mask
    task3 = ee.batch.Export.image.toDrive(
        image=fire_mask,
        description=f'{clean_name}_{alarm_date_short}',
        folder=f'{output_folder}/masks',
        scale=20,
        region=expand_regions,
        maxPixels=1e10,
        fileFormat='GeoTIFF'
    )
    task3.start()

    # Export dNBR
    task4 = ee.batch.Export.image.toDrive(
        image=dnbr,
        description=f'{clean_name}_{alarm_date_short}',
        folder=f'{output_folder}/dnbr',
        scale=20,
        region=expand_regions,
        maxPixels=1e10,
        fileFormat='GeoTIFF'
    )
    task4.start()

Processing region: KENNETH
Processing region: HURST
Processing region: LIDIA
Processing region: DOVE
Processing region: CLEAR
Processing region: FRENCH
Processing region: CANAL
Processing region: CODY
Processing region: LISA
Processing region: GROVE 2
Processing region: ROBLAR
Processing region: CREEK
Processing region: APACHE
Processing region: HERNANDEZ
Processing region: RANCH
Processing region: TULEDAD
Processing region: WATER
Processing region: VALLEY
Processing region: SKY
Processing region: CAMPBELL
Processing region: RECORD
Processing region: DORADO
Processing region: SHEERING
Processing region: STODDARD 3
Processing region: EL PASO
Processing region: POINT
Processing region: BELLA
Processing region: AMORUSO
Processing region: FLYNN
Processing region: HOLIDAY
Processing region: BLUE
Processing region: LUCERNE
Processing region: GLENHAVEN
Processing region: BOGUS
Processing region: POSTA 3
Processing region: BORDER 53
Processing region: PENCIL
Processing region: FELICIA
Processi

ConnectionError: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/587674934542/image:export?alt=json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F3AAD3C590>, 'Connection to earthengine.googleapis.com timed out. (connect timeout=None)'))